# Analog Input Calibration

Below are the following links relevant to this QUA program

[**Variables in QUA**](https://docs.quantum-machines.co/latest/docs/Guides/variables/?h=variable#variables-in-qua) can be integers, fixed point numbers, and booleans

[**Streaming Raw ADC results**](https://docs.quantum-machines.co/latest/docs/Guides/stream_proc/?h=adc_t#streaming-raw-adc-results) to stream out a raw ADC trace

[**Real-Time Global Reset Phase**](https://docs.quantum-machines.co/latest/docs/Introduction/qua_overview/?h=reset_phase#resetting-the-global-phase) sets the oscillator phase as to what it was at the beginning of the program

[**Wait QUA command**](https://docs.quantum-machines.co/latest/docs/API_references/qua/dsl_main/#qm.qua._dsl.wait) introduces delays to the timing of elements

[**Stream Processing Average**](https://docs.quantum-machines.co/latest/docs/API_references/qua/result_stream/?h=#qm.qua._dsl._ResultStream.average) are Stream Processing commands for data manipulation during the program execution

## QUA program

In [ ]:
from iqcc_cloud_client import IQCC_Cloud
import matplotlib.pyplot as plt
from importlib import reload
import qm.qua as qua
import configuration
reload(configuration)
from configuration import *

with qua.program() as analog_input_calibration:
    pass

qc = IQCC_Cloud() 
run_data = qc.execute(analog_input_calibration, config, False)

## Plotting

In [ ]:
# Fetch the raw ADC traces and convert them into Volts
adc1 = np.array(run_data['result']['adc1'])
adc2 = np.array(run_data['result']['adc2'])
adc1_single_run = np.array(run_data['result']['adc1_single_run'])
adc2_single_run = np.array(run_data['result']['adc2_single_run'])
# Plot data
plt.figure()
plt.subplot(121)
plt.title("Single run")
plt.plot(adc1_single_run, label="Input 1")
plt.plot(adc2_single_run, label="Input 2")
plt.xlabel("Time [ns]")
plt.ylabel("Signal amplitude [ADC]")
plt.legend()

plt.subplot(122)
plt.title("Averaged run")
plt.plot(adc1, label="Input 1")
plt.plot(adc2, label="Input 2")
plt.xlabel("Time [ns]")
plt.legend()
plt.tight_layout()

print(f"\nInput1 mean: {np.mean(adc1)/2**12} V\n" f"Input2 mean: {np.mean(adc2)/2**12} V")

# Resonator Spectroscopy

The following links to the documentation will help you write the QUA program to find the resonance frequency of the readout resonator.

[**Variables in QUA**](https://docs.quantum-machines.co/latest/docs/Guides/variables/?h=variable#variables-in-qua) can be integers, fixed point numbers, and booleans

[**Real-Time Frequency Update**](https://docs.quantum-machines.co/latest/docs/API_references/qua/dsl_main/#qm.qua._dsl.update_frequency) updates the oscillator's frequency associated with an element

[**Measure**](https://docs.quantum-machines.co/latest/docs/API_references/qua/dsl_main/?h=measure#qm.qua._dsl.measure) and [**Dual Demodulation**](https://docs.quantum-machines.co/latest/docs/Guides/demod/?h=dual#dual-demodulation) point you to waveform acquisition and manipulation

[**Wait QUA command**](https://docs.quantum-machines.co/latest/docs/API_references/qua/dsl_main/#qm.qua._dsl.wait) introduces delays to the timing of elements

[**QUA for loop**](https://docs.quantum-machines.co/latest/docs/API_references/qua/dsl_main/?h=#qm.qua._dsl.for_) is the instruction in QUA that mimics for-loops in CPUs

[**Save from Pulse Processing Unit to Stream Processing Unit**](https://docs.quantum-machines.co/latest/docs/API_references/qua/dsl_main/?h=save#qm.qua._dsl.save) and [**Declare Stream**](https://docs.quantum-machines.co/latest/docs/API_references/qua/dsl_main/?h=save#qm.qua._dsl.save) are the QUA commands related to data transfer within the OPX

[**Stream Processing Buffer**](https://docs.quantum-machines.co/latest/docs/API_references/qua/result_stream/?h=#qm.qua._dsl._ResultStream.buffer) and [**Stream Processing Average**](https://docs.quantum-machines.co/latest/docs/API_references/qua/result_stream/?h=#qm.qua._dsl._ResultStream.average) are Stream Processing commands for data manipulation during the program execution

## QUA program

You will be needing the following instructions, there are not complete to make it fun :)

Q = qua.declare(qua.fixed)

qua.update_frequency("resonator", f)

f = qua.declare(int)

I_st = qua.declare_stream()

qua.save(I, I_st)

qua.measure("readout", "resonator", None, qua.dual_demod.full("minus_sin", "out1", "cos", "out2", Q), ...)

In [ ]:
from iqcc_cloud_client import IQCC_Cloud
import matplotlib.pyplot as plt
from importlib import reload
import qm.qua as qua
import configuration
reload(configuration)
from configuration import *
from qualang_tools.loops import from_array

n_avg = 10  # The number of averages
# The frequency sweep parameters
f_min = 20 * u.MHz
f_max = 60 * u.MHz
df = 500 * u.kHz
frequencies = np.arange(f_min, f_max, df)

# HINT - use from_array to easily sweep over numpy array, e.g., with for_(*from_array(_qua_variable, _numpy_array))
with qua.program() as resonator_spectroscopy:
    pass

qc = IQCC_Cloud() 
run_data = qc.execute(resonator_spectroscopy, config, False)

## Plotting

In [ ]:
I = np.array(run_data['result']['I'])
Q = np.array(run_data['result']['Q'])

from scipy import signal
magnitude = np.sqrt(I**2 + Q**2)
phase = signal.detrend(np.unwrap(np.angle(I+1j*Q)))

plt.subplot(121)
plt.title('Magnitude')
plt.plot(frequencies, magnitude)
plt.subplot(122)
plt.title('Phase')
plt.plot(frequencies, phase)
plt.tight_layout()

# Resonator Spectroscopy vs Amplitude

In addition to the previous section links you will be needing the following to complete the QUA program

[**Real-Time Amplitude change**](https://docs.quantum-machines.co/latest/docs/API_references/qua/dsl_main/?h=amp#qm.qua._dsl.amp) will rescale the operation amplitude defined in the config dictionary

[**QUA for-each**](https://docs.quantum-machines.co/latest/docs/Guides/features/#for_each) to loop over arbitrary values

## QUA program

In [ ]:
from iqcc_cloud_client import IQCC_Cloud
import matplotlib.pyplot as plt
from importlib import reload
import qm.qua as qua
import configuration
reload(configuration)
from configuration import *

n_avg = 40  # The number of averages
# The frequency sweep around the resonator frequency
span = 10 * u.MHz
df = 100 * u.kHz
dfs = np.arange(-span, +span, df)
# The readout amplitude sweep (as a pre-factor of the readout amplitude) - must be within [-2; 2)
a_min = 0.001
a_max = 1.99
amplitudes = np.geomspace(a_min, a_max, 20)

# HINT - use with for_each_(_qua_variable, _python_list) to update the amplitude in log-scale
# HINT - use from_array to easily sweep over numpy array, e.g., with for_(*from_array(_qua_variable, _numpy_array))
# ->  with qua.for_each_(a, amplitudes.tolist()):


with qua.program() as resonator_spectroscopy_vs_amplitude:
    
    pass

qc = IQCC_Cloud()
run_data = qc.execute(resonator_spectroscopy_vs_amplitude, config, False)

## Plotting

In [ ]:
I = np.array(run_data['result']['I'])
Q = np.array(run_data['result']['Q'])
magnitude = np.sqrt(I**2 + Q**2)
magnitude /= magnitude.sum(axis=0)[np.newaxis, :]
plt.pcolor(amplitudes * readout_amp, dfs, magnitude)
plt.xscale('log')
plt.tight_layout()

# Multiplexed Resonators Readout

## QUA program

In [ ]:
from iqcc_cloud_client import IQCC_Cloud
import matplotlib.pyplot as plt
from importlib import reload
import qm.qua as qua
import configuration
reload(configuration)
from configuration import *
from qualang_tools.loops import from_array

n_avg = 10  # The number of averages
# The frequency sweep parameters
span = 10 * u.MHz
df = 500 * u.kHz
frequencies = np.arange(-span, span, df)

# HINT - use from_array to easily sweep over numpy array, e.g., with for_(*from_array(_qua_variable, _numpy_array))

with qua.program() as multiplexed_readout:

    pass


qc = IQCC_Cloud() 
run_data = qc.execute(multiplexed_readout, config, False)

## Plotting

In [ ]:
I1 = np.array(run_data['result']['I1'])
Q1 = np.array(run_data['result']['Q1'])
I2 = np.array(run_data['result']['I2'])
Q2 = np.array(run_data['result']['Q2'])

from scipy import signal
magnitude1 = np.sqrt(I1**2 + Q1**2)
phase1 = signal.detrend(np.unwrap(np.angle(I1+1j*Q1)))
magnitude2 = np.sqrt(I2**2 + Q2**2)
phase2 = signal.detrend(np.unwrap(np.angle(I2+1j*Q2)))

plt.subplot(221)
plt.title('Magnitude')
plt.plot(frequencies, magnitude1)
plt.subplot(223)
plt.title('Phase')
plt.plot(frequencies, phase1)
plt.subplot(222)
plt.title('Magnitude')
plt.plot(frequencies, magnitude2)
plt.subplot(224)
plt.title('Phase')
plt.plot(frequencies, phase2)
plt.tight_layout()

# Qubit spectroscopy

In [ ]:
from iqcc_cloud_client import IQCC_Cloud
import matplotlib.pyplot as plt
from importlib import reload
import qm.qua as qua
import configuration
reload(configuration)
from configuration import *
from qualang_tools.loops import from_array

n_avg = 100  # The number of averages
span = 150 * u.MHz
df = 1_000 * u.kHz
dfs = np.arange(-span, +span, df)

with qua.program() as qubit_spec:

    pass

qc = IQCC_Cloud() 
run_data = qc.execute(qubit_spec, config, False)

In [ ]:
I = np.array(run_data['result']['I'])
Q = np.array(run_data['result']['Q'])

from scipy import signal
magnitude = np.sqrt(I**2 + Q**2)
phase = signal.detrend(np.unwrap(np.angle(I+1j*Q)))

plt.subplot(121)
plt.title('Magnitude')
plt.plot(dfs, magnitude)
plt.subplot(122)
plt.title('Phase')
plt.plot(dfs, phase)
plt.tight_layout()

# Time Rabi

To build the time Rabi QUA program you will need to use Real-Time changes to [**Operation Duration**](https://docs.quantum-machines.co/latest/docs/Guides/features/?h=duration#dynamic-pulse-duration)

## QUA program

In [ ]:
from iqcc_cloud_client import IQCC_Cloud
import matplotlib.pyplot as plt
from importlib import reload
import qm.qua as qua
import configuration
reload(configuration)
from configuration import *
from qualang_tools.loops import from_array

n_avg = 10  # The number of averages
# Pulse duration sweep (in clock cycles = 4ns) - must be larger than 4 clock cycles
t_min = 16 // 4
t_max = 800 // 4
dt = 4 // 4
operation_durations = np.arange(t_min, t_max, dt)

# HINT - use from_array to easily sweep over numpy array, e.g., with for_(*from_array(_qua_variable, _numpy_array))

with qua.program() as time_rabi:
    
    pass

qc = IQCC_Cloud()
run_data = qc.execute(time_rabi, config, False)

## Plotting

In [ ]:
I = np.array(run_data['result']['I'])
Q = np.array(run_data['result']['Q'])
plt.subplot(121)
plt.plot(operation_durations * 4, I)
plt.subplot(122)
plt.plot(operation_durations * 4, Q)
plt.tight_layout()

# T1 experiment

# QUA program

In [ ]:
from iqcc_cloud_client import IQCC_Cloud
import matplotlib.pyplot as plt
from importlib import reload
import qm.qua as qua
import configuration
reload(configuration)
from configuration import *
from qualang_tools.loops import from_array

n_avg = 100
# Dephasing time sweep (in clock cycles = 4ns) - minimum is 4 clock cycles
tau_min = 16 // 4
tau_max = 600 // 4
d_tau = 8 // 4
operation_durations = np.arange(tau_min, tau_max + 0.1, d_tau)  # + 0.1 to add tau_max to taus

# HINT - use from_array to easily sweep over numpy array, e.g., with for_(*from_array(_qua_variable, _numpy_array))

with qua.program() as t1:
    pass

qc = IQCC_Cloud()
run_data = qc.execute(ramsey, config, False)

# Plotting

In [ ]:
I = np.array(run_data['result']['I'])
Q = np.array(run_data['result']['Q'])
plt.subplot(121)
plt.plot(operation_durations * 4, I)
plt.subplot(122)
plt.plot(operation_durations * 4, Q)
plt.tight_layout()

# Fit the results to extract the qubit decay time T1
try:
    from qualang_tools.plot.fitting import Fit

    fit = Fit()
    plt.figure()
    decay_fit = fit.T1(operation_durations * 4, I, plot=True)
    qubit_T1 = np.round(np.abs(decay_fit["T1"][0]) / 4) * 4
    plt.xlabel("Delay [ns]")
    plt.ylabel("I quadrature [V]")
    print(f"Qubit decay time to update in the config: qubit_T1 = {qubit_T1:.0f} ns")
    plt.legend((f"Relaxation time T1 = {qubit_T1:.0f} ns",))
    plt.title("T1 measurement")
except (Exception,):
    pass

# Frame Rotation Ramsey

To implemente this variant of Ramsey experiment you will need to use [**Real-Time Frame Rotations**](https://docs.quantum-machines.co/latest/docs/API_references/qua/dsl_main/?h=frame#qm.qua._dsl.frame_rotation_2pi)

## QUA program

In [ ]:
from iqcc_cloud_client import IQCC_Cloud
import matplotlib.pyplot as plt
from importlib import reload
import qm.qua as qua
import configuration
reload(configuration)
from configuration import *
from qualang_tools.loops import from_array

n_avg = 100
# Dephasing time sweep (in clock cycles = 4ns) - minimum is 4 clock cycles
tau_min = 16 // 4
tau_max = 600 // 4
d_tau = 8 // 4
operation_durations = np.arange(tau_min, tau_max + 0.1, d_tau)  # + 0.1 to add tau_max to taus
# Detuning converted into virtual Z-rotations to observe Ramsey oscillation and get the qubit frequency
detuning = 8 * u.MHz  # in Hz
delta_phase = (5 * u.MHz) * d_tau * 4 * 1e-09

# HINT - use from_array to easily sweep over numpy array, e.g., with for_(*from_array(_qua_variable, _numpy_array))

with qua.program() as ramsey:

    pass

qc = IQCC_Cloud()
run_data = qc.execute(ramsey, config, False)

## Plotting

In [ ]:
I = np.array(run_data['result']['I'])
Q = np.array(run_data['result']['Q'])
plt.subplot(121)
plt.plot(operation_durations * 4, I)
plt.subplot(122)
plt.plot(operation_durations * 4, Q)
plt.tight_layout()

# Active Reset

To implement the feedback capabilities for real-time active reset of the qubit state you will need [**Conditional Play**](https://docs.quantum-machines.co/latest/docs/Guides/features/?h=conditiona#conditional-play)

## QUA program

In [ ]:
from iqcc_cloud_client import IQCC_Cloud
import matplotlib.pyplot as plt
from importlib import reload
import qm.qua as qua
import workshop_configuration
reload(workshop_configuration)
from workshop_configuration import *
from qualang_tools.loops import from_array

n_runs = 1_000  # Number of runs

with qua.program() as active_reset:
    n = qua.declare(int)
    I_g = qua.declare(qua.fixed)
    Q_g = qua.declare(qua.fixed)
    I_g_st = qua.declare_stream()
    Q_g_st = qua.declare_stream()
    I_e = qua.declare(qua.fixed)
    Q_e = qua.declare(qua.fixed)
    I_e_st = qua.declare_stream()
    Q_e_st = qua.declare_stream()

    with qua.for_(n, 0, n < n_runs, n + 1):
        # Measure the state of the resonator
        qua.measure(
            "readout",
            "resonator",
            None,
            qua.dual_demod.full("rotated_cos", "rotated_sin", I_g),
            qua.dual_demod.full("rotated_minus_sin", "rotated_cos", Q_g),
        )
        # Wait for the qubit to decay to the ground state in the case of measurement induced transitions
        qua.wait(50_000, "resonator")
        # Save the 'I' & 'Q' quadratures to their respective streams for the ground state
        qua.save(I_g, I_g_st)
        qua.save(Q_g, Q_g_st)

        qua.align()  # global align
        # Play the x180 gate to put the qubit in the excited state
        qua.play("x180", "qubit")
        # Align the two elements to measure after playing the qubit pulse.
        qua.align("qubit", "resonator")
        # Measure the state of the resonator
        qua.measure(
            "readout",
            "resonator",
            None,
            qua.dual_demod.full("rotated_cos", "rotated_sin", I_e),
            qua.dual_demod.full("rotated_minus_sin", "rotated_cos", Q_e),
        )
        qua.wait(1000, 'qubit')
        qua.play('x180', 'qubit', condition = I_e > 1.037e-03)
        qua.align()
        qua.measure(
            "readout",
            "resonator",
            None,
            qua.dual_demod.full("rotated_cos", "rotated_sin", I_e),
            qua.dual_demod.full("rotated_minus_sin", "rotated_cos", Q_e),
        )
        qua.wait(50_000, "resonator")
        # Save the 'I' & 'Q' quadratures to their respective streams for the excited state
        qua.save(I_e, I_e_st)
        qua.save(Q_e, Q_e_st)

    with qua.stream_processing():
        # Save all streamed points for plotting the IQ blobs
        I_g_st.save_all("I_g")
        Q_g_st.save_all("Q_g")
        I_e_st.save_all("I_e")
        Q_e_st.save_all("Q_e")

qc = IQCC_Cloud()
run_data = qc.execute(active_reset, config, False)

## Plotting

In [ ]:
from qualang_tools.analysis.discriminator import two_state_discriminator

Ig = np.array(run_data['result']['I_g'])
Qg = np.array(run_data['result']['Q_g'])
Ie = np.array(run_data['result']['I_e'])
Qe = np.array(run_data['result']['Q_e'])

angle, threshold, fidelity, gg, ge, eg, ee = two_state_discriminator(Ig, Qg, Ie, Qe, b_print=True, b_plot=True)